In [1]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras import optimizers
from keras.utils import plot_model
from keras.models import Sequential, Model
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers import Dense, LSTM, RepeatVector, TimeDistributed, Flatten
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import tensorflow as tf

%matplotlib inline
warnings.filterwarnings("ignore")

# Set seeds to make the experiment more reproducible.
# import tensorflow as tf
# tf.random.set_seed() 
# from numpy.random import seed
# seed(1)

In [2]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, TimeDistributed, Conv1D, MaxPooling1D, Flatten, Bidirectional, Input, Flatten, Activation, Reshape, RepeatVector, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
train = pd.read_csv('train_time_data_new.csv') 
test = pd.read_csv('test_time_data_new.csv')
# train, test

In [4]:
train.drop(columns=['Stop_Time'], inplace=True)
test.drop(columns=['Stop_Time'], inplace=True)

In [5]:
l1 = ['Time_Since_Shift_Start','Time_Since_Morning','Station_Name_1-Stop_Reason_1-Remark_1',
 'Station_Name_1-Stop_Reason_3-Remark_12',
 'Station_Name_1-Stop_Reason_3-Remark_8',
 'Station_Name_10-Stop_Reason_4-Remark_4',
 'Station_Name_2-Stop_Reason_2-Remark_2',
 'Station_Name_3-Stop_Reason_3-Remark_10',
 'Station_Name_3-Stop_Reason_3-Remark_3',
 'Station_Name_3-Stop_Reason_3-Remark_6',
 'Station_Name_3-Stop_Reason_3-Remark_7',
 'Station_Name_3-Stop_Reason_4-Remark_4',
 'Station_Name_4-Stop_Reason_3-Remark_5',
 'Station_Name_4-Stop_Reason_4-Remark_4',
 'Station_Name_5-Stop_Reason_4-Remark_4',
 'Station_Name_6-Stop_Reason_3-Remark_13',
 'Station_Name_6-Stop_Reason_3-Remark_15',
 'Station_Name_6-Stop_Reason_4-Remark_4',
 'Station_Name_7-Stop_Reason_3-Remark_14',
 'Station_Name_7-Stop_Reason_3-Remark_9',
 'Station_Name_7-Stop_Reason_4-Remark_4',
 'Station_Name_8-Stop_Reason_3-Remark_11',
 'Station_Name_8-Stop_Reason_4-Remark_4',
 'Station_Name_9-Stop_Reason_4-Remark_4']

In [6]:
# train.drop(columns = ['index'],inplace=True)

In [5]:
from sklearn.preprocessing import MinMaxScaler
cpu_list = [i for i in train.columns if i != 'Stop_Time']
scaler = MinMaxScaler()
scaled_train = scaler.fit_transform(train[cpu_list])
scaled_test = scaler.transform(test[cpu_list])

In [6]:
def split_sequence(sequence, look_back, forecast_horizon):
 X, y = list(), list()
 for i in range(len(sequence)): 
   lag_end = i + look_back
   forecast_end = lag_end + forecast_horizon
   if forecast_end > len(sequence):
     break
   seq_x, seq_y = sequence[i:lag_end], sequence[lag_end:forecast_end]
   X.append(seq_x)
   y.append(seq_y)
 return np.array(X), np.array(y)

In [20]:
X

NameError: name 'X' is not defined

In [9]:
LOOK_BACK = 24    
FORECAST_RANGE = 4

In [10]:

n_features = len(cpu_list)
X_train, y_train = split_sequence(scaled_train, look_back=LOOK_BACK, forecast_horizon=FORECAST_RANGE)
X_test, y_test = split_sequence(scaled_test, look_back=LOOK_BACK, forecast_horizon=FORECAST_RANGE)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(5331, 24, 30)
(5331, 4, 30)
(2758, 24, 30)
(2758, 4, 30)


In [11]:
checkpoint_filepath = 'path_to_checkpoint_filepath'
checkpoint_callback = ModelCheckpoint(
 filepath=checkpoint_filepath,
 save_weights_only=False,
 monitor='val_loss',
 mode='min',
 save_best_only=True)
early_stopping_callback = EarlyStopping(
 monitor="val_loss",
 min_delta=0.005,
 patience=10,
 mode="min"
)
rlrop_callback = ReduceLROnPlateau(monitor='val_loss', factor=0.2, mode='min', patience=3, min_lr=0.001)

In [14]:
def inverse_transform(y_test, yhat):
 y_test_reshaped = y_test.reshape(-1, y_test.shape[-1])
 yhat_reshaped = yhat.reshape(-1, yhat.shape[-1])
 yhat_inverse = scaler.inverse_transform(yhat_reshaped) 
 y_test_inverse = scaler.inverse_transform(y_test_reshaped)
 return yhat_inverse, y_test_inverse

In [15]:
def evaluate_forecast(y_test_inverse, yhat_inverse):
 mse_ = tf.keras.losses.MeanSquaredError()
 mae_ = tf.keras.losses.MeanAbsoluteError()
 mape_ = tf.keras.losses.MeanAbsolutePercentageError()
 mae = mae_(y_test_inverse,yhat_inverse)
 print('mae:', mae)
 mse = mse_(y_test_inverse,yhat_inverse)
 print('mse:', mse)
 mape = mape_(y_test_inverse,yhat_inverse)
 print('mape:', mape)

In [14]:
epochs = 50
batch_size = 32
validation = 0.1

In [22]:
# input_layer = Input(shape=(LOOK_BACK, n_features)) 
# head_list = []
# for i in range(0, n_features):
#  conv_layer_head = Conv1D(filters=4, kernel_size=7, activation='relu')(input_layer)
#  conv_layer_head_2 = Conv1D(filters=6, kernel_size=11, activation='relu')(conv_layer_head)
#  conv_layer_flatten = Flatten()(conv_layer_head_2)
#  head_list.append(conv_layer_flatten)
 
# concat_cnn = Concatenate(axis=1)(head_list)
# reshape = Reshape((head_list[0].shape[1], n_features))(concat_cnn)
# lstm = LSTM(100, activation='tanh')(reshape)
# repeat = RepeatVector(FORECAST_RANGE)(lstm)
# # lstm_2 = LSTM(100, activation='tanh', return_sequences=True)(repeat)
# # dropout = Dropout(0.2)(lstm_2)
# dense = Dense(n_features, activation='linear')(dropout)
# multi_head_cnn_lstm_model = Model(inputs=input_layer, outputs=dense)
# multi_head_cnn_lstm_model.compile(optimizer='adam', loss='mse', metrics=['mse'])

In [24]:
input_layer = Input(shape=(LOOK_BACK, n_features)) 
head_list = []
for i in range(0, n_features):
 conv_layer_head = Conv1D(filters=64, kernel_size=1, activation='relu')(input_layer)
#  conv_layer_head_2 = Conv1D(filters=64, kernel_size=1, activation='relu')(conv_layer_head)
 conv_layer_flatten = Flatten()(conv_layer_head)#(conv_layer_head_2)
 head_list.append(conv_layer_flatten)   

#add pooling layer
# pool_layer = MaxPooling1D(pool_size=2)(input_layer)
 
concat_cnn = Concatenate(axis=1)(head_list)
reshape = Reshape((head_list[0].shape[1], n_features))(concat_cnn)
lstm = LSTM(100, activation='tanh')(reshape)
repeat = RepeatVector(FORECAST_RANGE)(lstm)
lstm_2 = LSTM(100, activation='tanh', return_sequences=True)(repeat)
dropout = Dropout(0.2)(lstm_2)
dense = Dense(n_features, activation='linear')(dropout)
multi_head_cnn_lstm_model = Model(inputs=input_layer, outputs=dense)
multi_head_cnn_lstm_model.compile(optimizer='adam', loss='mse', metrics=['mse'])

In [ ]:
#

In [25]:
plot_model(model=multi_head_cnn_lstm_model, show_shapes=True)
history = multi_head_cnn_lstm_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation,callbacks=[early_stopping_callback, checkpoint_callback, rlrop_callback])
yhat = multi_head_cnn_lstm_model.predict(X_test, verbose=0)
yhat_inverse, y_test_inverse = inverse_transform(y_test, yhat)
evaluate_forecast(y_test_inverse, yhat_inverse)

Epoch 1/50
186/186 [==============================] - ETA: 0s - loss: 0.0127 - mse: 0.0127

KeyboardInterrupt: 

In [18]:
num_samples = 6624

In [19]:
# import keras
# from keras.layers import Conv3D

# # Define the input shape for the data
# input_shape = (num_samples, 2, 22, 1)

# # Define the number of filters and the kernel size
# num_filters = 32
# kernel_size = (2, 22, 1)

# # Define the stride and padding
# stride = (1, 1, 1)
# padding = 'same'

# # Create the convolutional layer
# conv_layer = Conv3D(num_filters, kernel_size, strides=stride, padding=padding, input_shape=input_shape)




AttributeError: The layer "conv3d" has never been called and thus has no defined output shape.

In [ ]:
#save model with scaling normalization
multi_head_cnn_lstm_model.save('/content/drive/MyDrive/dataset/scaled_multi_head_cnn_lstm_model.h5')

In [ ]:
# plot_model(model=multi_head_cnn_lstm_model, show_shapes=True)
# history = multi_head_cnn_lstm_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation,callbacks=[checkpoint_callback, rlrop_callback])
# yhat = multi_head_cnn_lstm_model.predict(X_test, verbose=0)
# yhat_inverse, y_test_inverse = inverse_transform(y_test, yhat)
# evaluate_forecast(y_test_inverse, yhat_inverse)

In [ ]:
# evaluate_forecast(y_test_inverse, yhat_inverse)

mae: tf.Tensor(0.23641124, shape=(), dtype=float32)
mse: tf.Tensor(0.61856437, shape=(), dtype=float32)
mape: tf.Tensor(113829970.0, shape=(), dtype=float32)


In [25]:
multi_head_cnn_lstm_model.save('multi_head_cnn_lstm_model.h5') 

In [27]:
multi_head_cnn_lstm_model = tf.keras.models.load_model('multi_head_cnn_lstm_model.h5')

# Show the model architecture
multi_head_cnn_lstm_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 24, 24)]     0           []                               
                                                                                                  
 conv1d_48 (Conv1D)             (None, 18, 4)        676         ['input_2[0][0]']                
                                                                                                  
 conv1d_50 (Conv1D)             (None, 18, 4)        676         ['input_2[0][0]']                
                                                                                                  
 conv1d_52 (Conv1D)             (None, 18, 4)        676         ['input_2[0][0]']                
                                                                                            

In [15]:
test2 = pd.read_csv('trial_data_29_30.csv')

In [16]:
test2.drop(columns=['Stop_Time'],inplace=True)


In [17]:
test2.head()

,Time_Since_Morning,Station_Name_1-Stop_Reason_1-Remark_1,Station_Name_1-Stop_Reason_3-Remark_12,Station_Name_1-Stop_Reason_3-Remark_5,Station_Name_2-Stop_Reason_2-Remark_2,Station_Name_3-Stop_Reason_3-Remark_11,Station_Name_3-Stop_Reason_3-Remark_3,Station_Name_3-Stop_Reason_3-Remark_8,Station_Name_3-Stop_Reason_3-Remark_9,Station_Name_3-Stop_Reason_4-Remark_4,...,Station_Name_5-Stop_Reason_3-Remark_6,Station_Name_5-Stop_Reason_4-Remark_4,Station_Name_6-Stop_Reason_4-Remark_4,Station_Name_7-Stop_Reason_4-Remark_4,Station_Name_8-Stop_Reason_3-Remark_14,Station_Name_8-Stop_Reason_4-Remark_4,Station_Name_9-Stop_Reason_3-Remark_13,Station_Name_9-Stop_Reason_3-Remark_7,Station_Name_9-Stop_Reason_4-Remark_4,Time_Since_Shift_Start
0,1.0,7,0,0,5,0,4,0,0,1,...,0,1,0,0,0,0,0,0,0,1.0
1,2.0,17,0,3,6,0,1,0,0,1,...,1,0,3,2,0,0,0,0,0,2.0
2,3.0,11,0,0,6,0,0,0,0,1,...,0,2,3,2,0,2,0,0,0,3.0
3,4.0,7,0,0,6,0,0,0,0,1,...,0,0,2,1,0,1,0,0,0,4.0
4,5.0,6,0,3,6,0,0,0,0,0,...,0,2,4,0,0,0,0,0,1,5.0


In [18]:
train.shape

(6630, 24)

In [19]:
# test_data.shape

NameError: name 'test_data' is not defined

In [20]:
#if no of columns in test_data is more than len(l1) + 2, than remove the extra columns that are not in l1
cnt = 0
list_drop = []
if test_data.shape[1] > len(l1) + 2:
    for i in test_data.columns:
        if i not in l1:
            cnt += 1
            list_drop.append(i)
            if cnt == test_data.shape[1] - len(l1) - 2:
                break
cnt = 0
add_list = []
if test_data.shape[1] < len(l1) + 2:
    for i in l1:
        if i not in test_data.columns:
            add_list.append(i)
            cnt += 1
        if cnt == len(l1) + 2 - test_data.shape[1]:
            print('break')
            break
for i in add_list:
    test_data[i] = 0


        

NameError: name 'test_data' is not defined

In [54]:
test_data.shape

(35, 24)

In [55]:
from sklearn.preprocessing import MinMaxScaler
cpu_list = [i for i in test_data.columns if i != 'Stop_Time']
scaler = MinMaxScaler()
scaled_test = scaler.fit_transform(test_data[cpu_list])

In [ ]:
#predict on test data


In [51]:
#transform scaled_test to 3d array
scaled_test = scaled_test.reshape((scaled_test.shape[0], 1, scaled_test.shape[1]))
scaled_test.shape


(35, 1, 24)

In [50]:
yhat = multi_head_cnn_lstm_model.predict(scaled_test, verbose=0)
yhat_inverse, y_test_inverse = inverse_transform(y_test, yhat)
evaluate_forecast(y_test_inverse, yhat_inverse)

InvalidArgumentError: Graph execution error:

transpose expects a vector of size 3. But input(1) is a vector of size 4
	 [[{{node model_1/conv1d_48/Conv1D-0-TransposeNHWCToNCHW-LayoutOptimizer}}]] [Op:__inference_predict_function_203465]

In [ ]:
yhat_inverse

In [ ]:
y_test_inverse

array([[ 7., 10.,  0.,  0.,  6.,  0.,  0.,  1.,  0.,  0.,  1.,  4.,  2.,
         1.,  2.,  0.,  0.,  6.,  0.,  0.,  1.,  7.],
       [ 8., 14.,  0.,  0.,  6.,  0.,  0.,  0.,  0.,  2.,  0.,  6.,  0.,
         2.,  1.,  0.,  0.,  9.,  0.,  0.,  4.,  8.],
       [ 9., 13.,  0.,  0.,  6.,  0.,  0.,  1.,  1.,  1.,  1.,  5.,  0.,
         2.,  3.,  4.,  0.,  3.,  0.,  0.,  0.,  0.],
       [10., 13.,  0.,  0.,  6.,  0.,  0.,  0.,  1.,  2.,  0.,  7.,  0.,
         2.,  5.,  4.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 8., 14.,  0.,  0.,  6.,  0.,  0.,  0.,  0.,  2.,  0.,  6.,  0.,
         2.,  1.,  0.,  0.,  9.,  0.,  0.,  4.,  8.],
       [ 9., 13.,  0.,  0.,  6.,  0.,  0.,  1.,  1.,  1.,  1.,  5.,  0.,
         2.,  3.,  4.,  0.,  3.,  0.,  0.,  0.,  0.],
       [10., 13.,  0.,  0.,  6.,  0.,  0.,  0.,  1.,  2.,  0.,  7.,  0.,
         2.,  5.,  4.,  0.,  1.,  0.,  0.,  0.,  0.],
       [11., 10.,  0.,  0.,  6.,  0.,  0.,  0.,  1.,  2.,  0.,  1.,  1.,
         1.,  4.,  2.,  0.,  5.,  0., 

In [12]:
LOOK_BACK = 6    
FORECAST_RANGE = 1

In [16]:


n_features = len(cpu_list)
X_train, y_train = split_sequence(scaled_train, look_back=LOOK_BACK, forecast_horizon=FORECAST_RANGE)
X_test, y_test = split_sequence(scaled_test, look_back=LOOK_BACK, forecast_horizon=FORECAST_RANGE)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(5352, 6, 30)
(5352, 1, 30)
(2779, 6, 30)
(2779, 1, 30)


In [17]:
#create model with convolutional layer and pooling layer and lstm layer
input_layer = Input(shape=(X_train.shape[1], X_train.shape[2]))
conv_layer = Conv1D(filters=64, kernel_size=2, activation='relu')(input_layer)
pool_layer = MaxPooling1D(pool_size=2)(conv_layer)
lstm_layer = LSTM(50, activation='tanh')(pool_layer)
repeat_layer = RepeatVector(FORECAST_RANGE)(lstm_layer)
lstm_layer_2 = LSTM(50, activation='tanh', return_sequences=True)(repeat_layer)
dropout_layer = Dropout(0.2)(lstm_layer_2)
output_layer = Dense(n_features, activation='linear')(dropout_layer)
new_model = Model(inputs=input_layer, outputs=output_layer)
new_model.compile(optimizer='adam', loss='mse', metrics=['mse'])


In [ ]:
#plot model
plot_model(model=new_model, show_shapes=True)
history = new_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation,callbacks=[checkpoint_callback, rlrop_callback])
yhat = new_model.predict(X_test, verbose=0)
yhat_inverse, y_test_inverse = inverse_transform(y_test, yhat)
evaluate_forecast(y_test_inverse, yhat_inverse)


In [30]:
new_model.save('new_model.h5')

In [35]:
new_model.save('new_model_6.h5')

In [15]:
new_model_6 = tf.keras.models.load_model('new_model_6.h5')
print(new_model_6.summary())

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 6, 24)]           0         
                                                                 
 conv1d_98 (Conv1D)          (None, 5, 64)             3136      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 2, 64)            0         
 1D)                                                             
                                                                 
 lstm_5 (LSTM)               (None, 50)                23000     
                                                                 
 repeat_vector_3 (RepeatVect  (None, 1, 50)            0         
 or)                                                             
                                                                 
 lstm_6 (LSTM)               (None, 1, 50)             2020

In [18]:
    #plot model
plot_model(model=new_model, show_shapes=True)
history = new_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation,callbacks=[early_stopping_callback,checkpoint_callback, rlrop_callback])
yhat = new_model.predict(X_test, verbose=0)
yhat_inverse, y_test_inverse = inverse_transform(y_test, yhat)
evaluate_forecast(y_test_inverse, yhat_inverse)

Epoch 1/50
151/151 [==============================] - ETA: 0s - loss: 0.0187 - mse: 0.0187

INFO:tensorflow:Assets written to: path_to_checkpoint_filepath\assets


INFO:tensorflow:Assets written to: path_to_checkpoint_filepath\assets


151/151 [==============================] - 18s 58ms/step - loss: 0.0187 - mse: 0.0187 - val_loss: 0.0108 - val_mse: 0.0108 - lr: 0.0010
Epoch 2/50
147/151 [============================>.] - ETA: 0s - loss: 0.0098 - mse: 0.0098

INFO:tensorflow:Assets written to: path_to_checkpoint_filepath\assets


...

INFO:tensorflow:Assets written to: path_to_checkpoint_filepath\assets


151/151 [==============================] - 7s 50ms/step - loss: 0.0040 - mse: 0.0040 - val_loss: 0.0049 - val_mse: 0.0049 - lr: 0.0010
Epoch 14/50
151/151 [==============================] - 1s 9ms/step - loss: 0.0038 - mse: 0.0038 - val_loss: 0.0049 - val_mse: 0.0049 - lr: 0.0010
Epoch 15/50
147/151 [============================>.] - ETA: 0s - loss: 0.0037 - mse: 0.0037

INFO:tensorflow:Assets written to: path_to_checkpoint_filepath\assets


INFO:tensorflow:Assets written to: path_to_checkpoint_filepath\assets


151/151 [==============================] - 8s 50ms/step - loss: 0.0037 - mse: 0.0037 - val_loss: 0.0047 - val_mse: 0.0047 - lr: 0.0010
mae: tf.Tensor(0.472809, shape=(), dtype=float32)
mse: tf.Tensor(1.9031357, shape=(), dtype=float32)
mape: tf.Tensor(160639620.0, shape=(), dtype=float32)


In [19]:
new_model.save('new_model_early_stop.h5')


In [7]:
new_model_stop = tf.keras.models.load_model('new_model_early_stop.h5')
print(new_model_stop.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 6, 30)]           0         
                                                                 
 conv1d (Conv1D)             (None, 5, 64)             3904      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 2, 64)            0         
 )                                                               
                                                                 
 lstm (LSTM)                 (None, 50)                23000     
                                                                 
 repeat_vector (RepeatVector  (None, 1, 50)            0         
 )                                                               
                                                                 
 lstm_1 (LSTM)               (None, 1, 50)             20200 

In [8]:
test2 = pd.read_csv('test_data_21onw.csv')
test2.drop(columns=['Stop_Time'],inplace=True)
test2.head()

,Time_Since_Morning,Station_Name-Station_Name_1,Station_Name-Station_Name_2,Station_Name-Station_Name_3,Station_Name-Station_Name_4,Station_Name-Station_Name_5,Station_Name-Station_Name_6,Station_Name-Station_Name_7,Station_Name-Station_Name_8,Station_Name-Station_Name_9,...,Remarks-Remark_15,Remarks-Remark_2,Remarks-Remark_3,Remarks-Remark_4,Remarks-Remark_5,Remarks-Remark_6,Remarks-Remark_7,Remarks-Remark_8,Remarks-Remark_9,Time_Since_Shift_Start
0,1.0,11,6,4,2,4,4,1,2,0,...,0,6,15,1,1,0,0,0,0,1.0
1,2.0,15,6,5,8,4,3,0,8,1,...,0,6,23,0,6,0,0,0,0,2.0
2,3.0,10,6,0,2,2,1,0,2,4,...,0,6,11,0,0,0,0,0,0,3.0
3,4.0,7,6,0,2,0,1,0,0,4,...,0,6,6,0,1,0,0,0,0,4.0
4,5.0,6,6,1,3,2,4,1,2,3,...,0,6,11,3,0,2,0,0,0,5.0


In [9]:
test2.columns

Index(['Time_Since_Morning', 'Station_Name-Station_Name_1',
       'Station_Name-Station_Name_2', 'Station_Name-Station_Name_3',
       'Station_Name-Station_Name_4', 'Station_Name-Station_Name_5',
       'Station_Name-Station_Name_6', 'Station_Name-Station_Name_7',
       'Station_Name-Station_Name_8', 'Station_Name-Station_Name_9',
       'Stop_Reason-Stop_Reason_1', 'Stop_Reason-Stop_Reason_2',
       'Stop_Reason-Stop_Reason_3', 'Stop_Reason-Stop_Reason_4',
       'Remarks-Remark_1', 'Remarks-Remark_10', 'Remarks-Remark_11',
       'Remarks-Remark_12', 'Remarks-Remark_13', 'Remarks-Remark_14',
       'Remarks-Remark_15', 'Remarks-Remark_2', 'Remarks-Remark_3',
       'Remarks-Remark_4', 'Remarks-Remark_5', 'Remarks-Remark_6',
       'Remarks-Remark_7', 'Remarks-Remark_8', 'Remarks-Remark_9',
       'Time_Since_Shift_Start'],
      dtype='object')

In [54]:
# #if no of columns in test_data is more than len(l1) + 2, than remove the extra columns that are not in l1
# cnt = 0
# list_drop = []
# if test2.shape[1] > len(l1) :
#     for i in test2.columns:
#         if i not in l1:
#             cnt += 1
#             list_drop.append(i)
#             if cnt == test2.shape[1] - len(l1):
#                 break
# cnt = 0
# add_list = []
# if test2.shape[1] < len(l1):
#     for i in l1:
#         if i not in test2.columns:
#             add_list.append(i)
#             cnt += 1
#         if cnt == len(l1) - test2.shape[1]:
#             print('break')
#             break
# for i in add_list:
#     test2[i] = 0


break


In [10]:
#scale test data
scaled_test2 = scaler.transform(test2[cpu_list])

In [16]:
#prepare test2 for predict using split_sequence
x_test2, y_test2 = split_sequence(test2, look_back=LOOK_BACK, forecast_horizon=FORECAST_RANGE)
#predict on test2
yhat = new_model_stop.predict(x_test2, verbose=0)
yhat_inverse, y_test_inverse = inverse_transform(y_test2, yhat)
evaluate_forecast(y_test_inverse, yhat_inverse)




mae: tf.Tensor(77.48452, shape=(), dtype=float32)
mse: tf.Tensor(19796.516, shape=(), dtype=float32)
mape: tf.Tensor(390702720.0, shape=(), dtype=float32)


In [41]:
print(x_test2.shape)
print(y_test2.shape)

(159, 6, 30)
(159, 1, 30)


In [36]:
#convert x_test2 from 3d array to 2d array
x_test2 = x_test2.reshape((x_test2.shape[0], x_test2.shape[1] * x_test2.shape[2]))
print(x_test2.shape)

(159, 180)


In [17]:
yhat

array([[[ 0.3056625 ,  0.23215991,  1.2370276 , ...,  0.0201186 ,
          0.00797239,  0.07535709]],

       [[ 0.2712811 ,  0.06535969,  1.0291069 , ..., -0.00316954,
          0.0056748 , -0.08730485]],

       [[ 0.20258322,  0.09058561,  0.95129424, ..., -0.00530963,
          0.0091281 ,  0.0114034 ]],

       ...,

       [[ 0.78716373, -0.13171768,  0.8103206 , ..., -0.02368183,
          0.0095648 ,  0.875703  ]],

       [[ 0.96636075, -0.16159508,  0.96571714, ..., -0.02656214,
          0.00711047,  1.1742175 ]],

       [[ 1.0857854 , -0.11674834,  0.9818517 , ..., -0.02470803,
          0.00936075,  1.4690292 ]]], dtype=float32)

In [18]:
yhat_inverse

array([[ 7.03023767e+00,  5.10751820e+00,  5.07700233e+01, ...,
         1.40830189e-01,  1.19585842e-01,  1.13035643e+00],
       [ 6.23946524e+00,  1.43791318e+00,  4.30769539e+01, ...,
        -2.21867487e-02,  8.51220116e-02, -1.30957270e+00],
       [ 4.65941429e+00,  1.99288344e+00,  4.01978874e+01, ...,
        -3.71673778e-02,  1.36921450e-01,  1.71050951e-01],
       ...,
       [ 1.81047668e+01, -2.89778900e+00,  3.49818611e+01, ...,
        -1.65772811e-01,  1.43471956e-01,  1.31355448e+01],
       [ 2.22262974e+01, -3.55509162e+00,  4.07315331e+01, ...,
        -1.85934946e-01,  1.06657065e-01,  1.76132622e+01],
       [ 2.49730644e+01, -2.56846356e+00,  4.13285141e+01, ...,
        -1.72956228e-01,  1.40411213e-01,  2.20354385e+01]], dtype=float32)

In [19]:
#descaling yhat_inverse
yhat_inverse = scaler.inverse_transform(yhat_inverse)
yhat_inverse

array([[ 1.6169547e+02,  1.1236540e+02,  1.8834908e+03, ...,
         9.8581135e-01,  1.7937876e+00,  1.6955347e+01],
       [ 1.4350771e+02,  3.1634090e+01,  1.5988473e+03, ...,
        -1.5530723e-01,  1.2768302e+00, -1.9643591e+01],
       [ 1.0716653e+02,  4.3843437e+01,  1.4923219e+03, ...,
        -2.6017165e-01,  2.0538218e+00,  2.5657642e+00],
       ...,
       [ 4.1640964e+02, -6.3751358e+01,  1.2993289e+03, ...,
        -1.1604097e+00,  2.1520793e+00,  1.9703317e+02],
       [ 5.1120483e+02, -7.8212013e+01,  1.5120668e+03, ...,
        -1.3015447e+00,  1.5998560e+00,  2.6419894e+02],
       [ 5.7438049e+02, -5.6506199e+01,  1.5341550e+03, ...,
        -1.2106936e+00,  2.1061683e+00,  3.3053159e+02]], dtype=float32)

In [ ]:
#denormalizing yhat_inverse
